# Natural Language Processing: Detecting Humor
## TP2 - IART - G44
- [Afonso Caiado, up201806789](https://sigarra.up.pt/feup/pt/fest_geral.cursos_list?pv_num_unico=201806789)
- [Diogo Nunes, up201808546](https://sigarra.up.pt/feup/pt/fest_geral.cursos_list?pv_num_unico=201808546)
- [João Pinto, up201806667](https://sigarra.up.pt/feup/pt/fest_geral.cursos_list?pv_num_unico=201806667)

**It is recommended to [view this notebook in nbviewer](http://nbviewer.ipython.org/github/rhiever/Data-Analysis-and-Machine-Learning-Projects/blob/master/example-data-science-notebook/Example%20Machine%20Learning%20Notebook.ipynb) for the best viewing experience.**

**You can also [execute the code in this notebook on Binder](https://mybinder.org/v2/gh/rhiever/Data-Analysis-and-Machine-Learning-Projects/master?filepath=example-data-science-notebook%2FExample%20Machine%20Learning%20Notebook.ipynb) - no local installation required.**

## Table of contents

1. [Introduction](#Introduction)

2. [Required libraries](#Required-libraries)

3. [Main](#Main)

11. [Conclusions](#Conclusions)

In [40]:
import pandas as pd
fields = ['id', 'is_humor', 'humor_rating', 'text']

# Importing the test dataset
dataset = pd.read_csv('datasets/hahackathon_train.csv', index_col=False, usecols=fields)['id', ]

print(dataset)

        id                                               text  is_humor  \
0        1  TENNESSEE: We're the best state. Nobody even c...         1   
1        2  A man inserted an advertisement in the classif...         1   
2        3  How many men does it take to open a can of bee...         1   
3        4  Told my mom I hit 1200 Twitter followers. She ...         1   
4        5  Roses are dead. Love is fake. Weddings are bas...         1   
...    ...                                                ...       ...   
7995  7996  Lack of awareness of the pervasiveness of raci...         0   
7996  7997    Why are aspirins white? Because they work sorry         1   
7997  7998  Today, we Americans celebrate our independence...         1   
7998  7999  How to keep the flies off the bride at an Ital...         1   
7999  8000  "Each ounce of sunflower seeds gives you 37% o...         0   

      humor_rating  
0             2.42  
1             2.50  
2             1.95  
3             2